In [5]:
import tensorflow as tf

PATCH_SIZE = 7
PATCH_PIXELS = 7*7

BATCH_SIZE = 100

NUM_THREADS = 4

def read(queue):
    reader = tf.TFRecordReader()
    _, example = reader.read(queue)
    
    features = tf.parse_single_example(example, features={
        'us': tf.FixedLenFeature([], tf.string),
        'mr': tf.FixedLenFeature([], tf.string),
    })
    
    image = tf.decode_raw(features['mr'], tf.float64)
    label = tf.decode_raw(features['us'], tf.float64)
    
    image.set_shape([PATCH_SIZE])
    label.set_shape([PATCH_SIZE])
    
    return image, label

def inputs(filenames):
    queue = tf.train.string_input_producer(filenames)
    
    return tf.train.shuffle_batch(
        read(queue),
        batch_size=BATCH_SIZE,
        num_threads=NUM_THREADS,
        capacity=1000 + 3*BATCH_SIZE,
        min_after_dequeue=1000)

In [6]:
# weights and biases
w = tf.Variable(tf.zeros([PATCH_PIXELS, PATCH_PIXELS]))
b = tf.Variable(tf.zeros([PATCH_PIXELS]))

# placeholders to be fed with images, labels batch
x = tf.placeholder(tf.float64, shape=[BATCH_SIZE, PATCH_PIXELS])
y = tf.placeholder(tf.float64, shape=[BATCH_SIZE, PATCH_PIXELS])

# prediction function
f = tf.matmul(w, x) + b

# cost function
c = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(f, y))

# train step
t = tf.train.GradientDescentOptimizer(.5).minimize(c)

TypeError: Input 'b' of 'MatMul' Op has type float64 that does not match type float32 of argument 'a'.

In [ ]:
import time

test_files = ['{:02}.tfrecord'.format(i) for range(1, 8)]
train_files = ['{:02}.tfrecord'.format(i) for range(8, 15)]

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        step = 0
        
        while not coord.should_stop():
            start_time = time.time()
            
            # sess.run(train_step)
            
            duration = start_time - time.time()
            
            if step % 100 == 0:
                print('step {}: '.format(step))
                
            step += 1
    except tf.errors.OutOfRangeError:
        print('done with training')
    finally:
        coord.request_stop()
        coord.join(threads)